In [1]:
import xmltodict
import os
import numpy as np
import cv2

xml_path = '。/results.xml'
sample_path = './sample'
result_path = './result'


In [2]:
with open(xml_path, 'r') as file:
        data = file.read().replace('\n', '')
        
xml_dict=xmltodict.parse(data)

In [ ]:
print(xml_dict['results']['image'][0]['object']['bndbox'])
print(xml_dict['results']['image'][0]['filename'])

In [4]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou


In [ ]:
iou_list = []
detected = 0
for image_result in xml_dict['results']['image']:
    filename = image_result['filename']
    bbox = [image_result['object']['bndbox']['xmin'],image_result['object']['bndbox']['ymin'],image_result['object']['bndbox']['xmax'],image_result['object']['bndbox']['ymax']]
    bbox = [int(float(box)) for box in bbox]
    bbox = np.array(bbox, dtype = np.float32)   
    label_path = os.path.join(sample_path, filename.split('.jpg')[0] + '.xml')
    with open(label_path, 'r') as file:
        data = file.read().replace('\n', '')
    xml=xmltodict.parse(data)
    xmax = int(xml['annotation']['object']['bndbox']['xmax'])
    xmin = int(xml['annotation']['object']['bndbox']['xmin'])
    ymax = int(xml['annotation']['object']['bndbox']['ymax'])
    ymin = int(xml['annotation']['object']['bndbox']['ymin'])
    xcenter = (xmax + xmin) / 2
    ycenter = (ymax + ymin) / 2
    xlen = xmax - xmin
    ylen = ymax - ymin   
    target = np.array([xmin,ymin,xmax,ymax], dtype = np.float32)   
    
    
    iou = bb_intersection_over_union(bbox,target)#bbox_iou(bbox.reshape(1,4),target.reshape(1,4),False)
    print(filename)
    print('bbox',bbox)
    print('target',target)
    print('iou',iou)
    iou_list.append(iou)

    img = cv2.imread(os.path.join(sample_path, filename), cv2.IMREAD_COLOR)
    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0,255,0), 2)
    cv2.imwrite(os.path.join(result_path, filename), img)

    if iou>0.5:
        detected += 1

print('iou',np.mean(iou_list))
print('detected',detected)